In [1]:
from libraries import *

In [ ]:
df = pd.read_csv("1-15Data-cleaned.csv")
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased") # nlpaueb/legal-bert-base-uncased   |  
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
import numpy as np
import pandas as pd

# Assuming df is your DataFrame with long text in 'Content' column
chunk_size = 512  # or the max length of your model
final_embeddings = []

for content in df['Content']:
    # Step 1: Split the content into chunks
    try:
        chunks = [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]
        
        # Step 2: Get embeddings for each chunk
        chunk_embeddings = []
        for chunk in chunks:
            inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True, max_length=chunk_size).to(device)
            with torch.no_grad():
                chunk_embedding = model(**inputs).last_hidden_state.mean(dim=1)  # Mean pooling
                chunk_embeddings.append(chunk_embedding.cpu().numpy())
        
        # Step 3: Combine chunk embeddings (mean pooling)
        final_embedding = np.mean(chunk_embeddings, axis=0)  # Mean pooling
        final_embeddings.append(final_embedding)
    except Exception as e:
        print(e)

# Step 4: Add final embeddings to DataFrame
df['Final_Embedding'] = final_embeddings


In [11]:
len(final_embeddings)

145

In [16]:
df = df.drop(47)
df = df.reset_index(drop=True)
len(df)

145

In [17]:
df['Final_Embedding'] = final_embeddings

In [19]:
df['Final_Embedding'].to_csv('test-embedding-withpooling.csv')

In [20]:
df['Final_Embedding'].shape

(145,)